# Mock for Song

## Notes

We work in distances of **Mpc/h** and masses of **Msun**.

We want to take an N-body simulation (currently MDPL but could be anything) and put galaxies in it in a way that creates a universe consistent with HSC observations. We are building an HSC mock catalog.

To do this we use the functional form which relates stellar mass to halo mass from [Behroozi2010](https://arxiv.org/pdf/1001.0015.pdf). As this is a mean relation we will also need a scatter term which we allow to vary linearly with log(halo mass). Finally, we get a good starting fit for the Behroozi functional form from [Leauthaud2012](https://arxiv.org/pdf/1104.0928.pdf) and for the scatter term from CB in prep.

As this functional form relates stellar mass - halo mass we initially generated stellar masses using various halo masses (Mvir, Mpeak). We were able to fit the SMF but clustering was low. After talking to Peter, this is apparently known and we should instead be populating stellar mass using Vmax@Mpeak. See [appendix A](./appendix.ipynb) for a description of how we get the initial coordinates.

With these initial fits we run an optimizer that takes into account both the SM and the clustering. We find that we can match the SMF well (all bins within the error bars) but have low clustering (HSC is 18.5 +/- 1.7, mock is 14 +/- 0.2). I don't really know how to evaluate whether this is good enough.

### How to run this

* If you want to re-run the opimizer, just run everything though  (takes O(hours)).
* If you just want to build the best fit model, run everything except the optimizer (takes O(1 minute)).

## Import etc...

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.interpolate, scipy.optimize
import emcee

import data as d
import likelihood as l
import plots as p
import clustering as c

import smhm_fit
from get_sm_for_sim import get_sm_for_sim, get_smf

from Corrfunc.utils import convert_3d_counts_to_cf

%run ~/.jupyter/config.ipy

# Load data + initial config

In [2]:
# For the clustering, we look around objects with mass greater than this and
# count objects in the 0.1 dex smaller than it.
cen_sat_div = [11.65, 11.75, 11.85]

# See the appendix A for how we got these starting params
b_params = [2.4, 10.91, 0.6, 0.2, 0.2]
s_params = [-0.1, 0.5]
params = b_params + s_params
x_field = "Vmax@Mpeak"

# To speed things up we store a previous best_fit
best_fit = params = [ 2.37120547, 10.92851565,  0.34209756,  0.21683624,  0.25941454, -0.10938057,  0.51421793]

In [3]:
# The simulation data
sim_data, sim_size = d.load_mdpl()

# The observational data that we fit to
hsc_gals = d.load_hsc_gals()
obs_clust = np.array(
    [c.compute_hsc_clustering(hsc_gals, div) for div in cen_sat_div],
    dtype=[("clustering", np.float64), ("err", np.float64)])
obs_smf = d.load_smf()

/home/christopher/.virtualenvs/greg/lib/python3.6/site-packages/Corrfunc/mocks/DDrppi_mocks.py:348: UserWarning: One or more input array has non-native endianness!  A copy will be made with the correct endianness.
  warn('One or more input array has non-native endianness!  A copy will be made with the correct endianness.')


# Find bestfit to SMF with optimizer

In [ ]:
res = scipy.optimize.minimize(
    l.compute_chi2_n,
    x0 = params,
    args=(sim_data, obs_smf, obs_clust, sim_size, cen_sat_div, x_field, 1),
    method="Nelder-Mead",
    tol=0.1,
)
assert res.success == True
best_fit = res.x

[ 2.37120547 10.92851565  0.34209756  0.21683624  0.25941454 -0.10938057  0.51421793]
This is not good [5702] [5747]
This is not good [1111] [1479]
This is not good [151] [285]
1.787038185622463
[ 2.48976574 10.92851565  0.34209756  0.21683624  0.25941454 -0.10938057  0.51421793]
This is not good [511] [794]
This is not good [779] [794]
This is not good [74] [148]
This is not good [8] [22]
This is not good [17] [22]
250.88244396226372
[ 2.37120547 11.47494143  0.34209756  0.21683624  0.25941454 -0.10938057  0.51421793]
656576.067682723
[ 2.37120547 10.92851565  0.35920244  0.21683624  0.25941454 -0.10938057  0.51421793]
This is not good [5237] [5249]
This is not good [1035] [1321]
This is not good [139] [259]
2.1441597672252533
[ 2.37120547 10.92851565  0.34209756  0.22767805  0.25941454 -0.10938057  0.51421793]
This is not good [1033] [1296]
This is not good [153] [275]
This is not good [259] [275]
2.2781057461412337
[ 2.37120547 10.92851565  0.34209756  0.21683624  0.27238527 -0.1093

This is not good [1061] [1336]
This is not good [167] [284]
1.5170396114250222
[ 2.37111269 10.93026838  0.34844579  0.21908993  0.26422844 -0.10874297  0.50889272]
This is not good [5043] [5210]
This is not good [980] [1362]
This is not good [136] [258]
3.516798613813955
[ 2.37293841 10.92878766  0.34560817  0.21861889  0.26207666 -0.10965682  0.5181833 ]
This is not good [5653] [5697]
This is not good [1147] [1399]
This is not good [156] [311]
1.880895789202421
[ 2.37394932 10.93012454  0.33002157  0.2212249   0.2669339  -0.10940276  0.51708791]
This is not good [1204] [1532]
This is not good [177] [328]
This is not good [327] [328]
1.8453618567994559
[ 2.37473328 10.93058423  0.34367617  0.22247881  0.25240564 -0.10940911  0.51790791]
This is not good [5869] [5924]
This is not good [1150] [1435]
This is not good [210] [289]
2.6601320782155056
[ 2.37208742 10.92903279  0.34249221  0.21824688  0.26739036 -0.1093877   0.51514042]
This is not good [1138] [1373]
This is not good [162] [2

This is not good [182] [275]
1.6879864350132645
[ 2.37371323 10.92956193  0.34286372  0.2217136   0.25854138 -0.1089751   0.51509112]
This is not good [5518] [5556]
This is not good [1111] [1433]
This is not good [178] [294]
1.1884470903430722
[ 2.37392582 10.92885417  0.34230651  0.22216777  0.25851958 -0.10876266  0.514958  ]
This is not good [1067] [1411]
This is not good [195] [268]
1.524857156815981
[ 2.37318899 10.93007618  0.34287502  0.22169769  0.2585385  -0.10902153  0.51502212]
This is not good [1104] [1410]
This is not good [179] [294]
1.4469737058610452
[ 2.37339032 10.93020023  0.34198242  0.22168445  0.2580877  -0.10928594  0.51481714]
This is not good [1095] [1387]
This is not good [169] [322]
This is not good [321] [322]
1.667268553163277
[ 2.37347263 10.93014376  0.34366646  0.22130648  0.25871153 -0.10907802  0.5152828 ]
This is not good [1108] [1427]
This is not good [159] [325]
1.1729593530265197
[ 2.37330449 10.92991111  0.34376951  0.22168093  0.25800547 -0.10901

This is not good [1087] [1429]
This is not good [183] [282]
1.4257776103439337
[ 2.37357744 10.93012677  0.34330379  0.22137512  0.25824556 -0.10903952  0.51529411]
This is not good [5576] [5657]
This is not good [1139] [1403]
This is not good [213] [271]
2.188364991583774
[ 2.37359733 10.93007266  0.34343039  0.22137336  0.25817246 -0.10902275  0.51533049]
This is not good [1126] [1456]
This is not good [158] [298]
This is not good [296] [298]
1.1869306354363443
[ 2.37358763 10.93007173  0.34346248  0.22137715  0.25820034 -0.10903206  0.51535854]
This is not good [1110] [1418]
This is not good [137] [288]
1.5002904106339048
[ 2.37359283 10.93006327  0.34340842  0.22136569  0.25825578 -0.10902587  0.51533849]
This is not good [1107] [1392]
This is not good [185] [308]
1.686153829782604
[ 2.37357484 10.93006481  0.34341913  0.22139285  0.25821774 -0.10903088  0.51533793]
This is not good [1086] [1388]
This is not good [173] [277]
1.6015046383970595
[ 2.37360116 10.93005822  0.34341135  

# Build bestfit model and compare to observations

In [ ]:
log_stellar_masses = get_sm_for_sim(sim_data, best_fit[:5], best_fit[5:], x_field)

In [ ]:
ax = p.plot_smfs(sim_data, obs_smf, best_fit[:5], best_fit[5:], sim_size, x_field)
ax.set(ylim=(1e-8, 1e-2));

sim_clust = np.array(
    [c.compute_sim_clustering(sim_data, sim_size, log_stellar_masses, div) for div in cen_sat_div],
    dtype=[("clustering", np.float64), ("err", np.float64)])
print(obs_clust)
print(sim_clust)

# Save out mock

In [ ]:
sim_data_out = np.lib.recfunctions.append_fields(sim_data, names="stellar_mass", dtypes=np.float64, data=np.power(10, log_stellar_masses))

In [ ]:
sim_data_out = np.lib.recfunctions.rename_fields(sim_data_out, {"halo_mvir": "Mpeak"})

In [ ]:
np.save("mock", np.ma.getdata(sim_data_out))